[View in Colaboratory](https://colab.research.google.com/github/FullteaOfEEIC/oMap/blob/cnn/cnn.ipynb)

In [15]:
#settings and import libraries.


#TensorFlow
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from matplotlib.pyplot import imshow
print(tf.test.gpu_device_name())

#Google Drive
#Notes. Colaboratory may requires you a permission to access Google Drive.
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/oMap
%pwd

#matplot
%matplotlib inline
tf.test.gpu_device_name()

#tqdm
from tqdm import tqdm

#others
import cv2
import numpy as np
import random
import os
import shutil

/device:GPU:0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/oMap'
/content/drive/My Drive/oMap


In [0]:
#globals

num_classes = 2
width,height=300,300
shape = (width,height,3) #RGB,300*300
epochs=40
batch_size=64

In [0]:
def clearFolder(path):
  if os.path.isdir(path):
    contents=os.listdir(path)
    for content in contents:
      try:
        shutil.rmtree(os.path.join(path,content))
      except NotADirectoryError:
        os.remove(os.path.join(path,content))
  else:
    os.makedirs(path)

In [0]:
#Functions in order to trim the images of posts.

def postMap(all, white):
    post = 255 - (imgWhite - imgAll)
    post = cv2.cvtColor(post, cv2.COLOR_BGR2GRAY)
    ret, post = cv2.threshold(post, 200, 255, cv2.THRESH_BINARY)
    return post

  
def getPost(imgPost, comp):
    circles = cv2.HoughCircles(imgPost, cv2.HOUGH_GRADIENT, 1,
                               20, param1=50, param2=30, minRadius=95, maxRadius=125)
    imgPost = cv2.medianBlur(imgPost, 3)
    circles = np.uint16(np.around(circles))
    imgPost = cv2.cvtColor(imgPost, cv2.COLOR_GRAY2BGR)

    #create an img which only the position of posts is drawn.
    for i in circles[0, :]:
        cv2.circle(imgPost, (i[0], i[1]), 2, (0, 0, 255), 3)
        cv2.circle(imgPost, (i[0], i[1]), i[2], (0, 255, 0), 2)
    cv2.imwrite("detected{}.png".format(comp), imgPost)

    return circles[0, :]

  
def trim(img, centerX, centerY, width, height, name,save=True):
    leftTop = (centerX - int(width / 2), centerY - int(height / 2))
    imgTrim = img[leftTop[1]:leftTop[1] +
                  height, leftTop[0]:leftTop[0] + width]
    if save:
        cv2.imwrite(name, imgTrim)
    return imgTrim


def inflation(folder,out):
    files=os.listdir(folder)
    for file in files:
        if file[-4::]==(".png" or ".jpg"):
            img=cv2.imread(os.path.join(folder,file))
            for i in range(4):
                img=img.transpose(1,0,2)[:,::-1]
                cv2.imwrite(os.path.join(out,file)[:-4:]+"-{}.png".format(90*i),img)
                imgMirror=cv2.flip(img,1)
                cv2.imwrite(os.path.join(out,file)[:-4:]+"-{}-t.png".format(90*i),imgMirror)




In [19]:
clearFolder("Post")
clearFolder("NOTPost")
clearFolder("Post水増し")
clearFolder("NOTPost水増し")

for comp in (38, 39):
    imgAll = cv2.imread("all{0}.jpg".format(comp))
    imgWhite = cv2.imread("white{0}.jpg".format(comp))
    imgPost = postMap(imgAll, imgWhite)
    cv2.imwrite("thresh{}.png".format(comp),imgPost)
    posts = getPost(imgPost, comp)
    for i, post in tqdm(enumerate(posts)):
        trim(imgWhite, post[0], post[1], width, height,"Post/{0}post-{1}.png".format(comp, i))
        trim(imgWhite, random.randint(posts.min(axis=0)[0], posts.max(axis=0)[0]), random.randint(
                posts.min(axis=0)[1], posts.max(axis=0)[1]), width, height, "NOTPost/{0}not-{1}.png".format(comp, i))

inflation("Post","Post水増し")
inflation("NOTPost","NOTPost水増し")
print("Done")
    

38it [00:01, 36.17it/s]
48it [00:00, 48.20it/s]


Done


In [20]:
imgTrue=os.listdir("Post水増し")
imgFalse=os.listdir("NOTPost水増し")
size=len(imgTrue)# = len(imgFalse)
print(size)

imgTrain=[]
labelTrain=[]
imgTest=[]
labelTest=[]

for img in imgTrue[:int(size*0.8):]:
  path=os.path.join("Post水増し",img)
  imgTrain.append(cv2.imread(path)/255)
  labelTrain.append([1,0])
for img in imgFalse[:int(size*0.8):]:
  path=os.path.join("NOTPost水増し",img)
  imgTrain.append(cv2.imread(path)/255)
  labelTrain.append([0,1])
  
for img in imgTrue[int(size*0.8)::]:
  path=os.path.join("Post水増し",img)
  imgTest.append(cv2.imread(path)/255)
  labelTest.append([1,0])
for img in imgFalse[int(size*0.8)::]:
  path=os.path.join("NOTPost水増し",img)
  imgTest.append(cv2.imread(path)/255)
  labelTest.append([0,1])

imgTrain=np.asarray(imgTrain,dtype="float32")
imgTest=np.asarray(imgTest,dtype="float32")
labelTrain=np.asarray(labelTrain,dtype=np.int)
labelTest=np.asarray(labelTest,dtype=np.int)


688


In [23]:
def cnn():
  model = Sequential()
  model.add(Conv2D(16,3,padding="same",input_shape=shape))
  model.add(Activation("relu"))
  model.add(Conv2D(16,3,padding="same"))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(16, 32, padding='same'))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation("relu"))
  model.add(Dropout(0.8))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  
  return model
  
  
model = cnn()
model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=1e-4),metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 300, 300, 16)      448       
_________________________________________________________________
activation_43 (Activation)   (None, 300, 300, 16)      0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 300, 300, 16)      2320      
_________________________________________________________________
activation_44 (Activation)   (None, 300, 300, 16)      0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 150, 150, 16)      0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 150, 150, 16)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 150, 150, 16)      262160    
__________

In [25]:
history=model.fit(imgTrain,labelTrain,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(imgTest,labelTest))
model.save("oMap.h5")

Train on 1100 samples, validate on 276 samples
Epoch 1/40
1100/1100 [==============================] - 9s 8ms/step - loss: 0.6727 - acc: 0.5618 - val_loss: 0.7884 - val_acc: 0.4529
Epoch 2/40
1100/1100 [==============================] - 9s 8ms/step - loss: 0.6637 - acc: 0.5709 - val_loss: 0.7635 - val_acc: 0.4855
Epoch 3/40
1100/1100 [==============================] - 9s 8ms/step - loss: 0.6628 - acc: 0.5664 - val_loss: 0.7832 - val_acc: 0.4601
Epoch 4/40
1100/1100 [==============================] - 9s 8ms/step - loss: 0.6521 - acc: 0.5627 - val_loss: 0.8273 - val_acc: 0.4601
Epoch 5/40
1100/1100 [==============================] - 9s 8ms/step - loss: 0.6496 - acc: 0.5636 - val_loss: 0.9418 - val_acc: 0.4239
Epoch 6/40
1100/1100 [==============================] - 9s 8ms/step - loss: 0.6335 - acc: 0.6000 - val_loss: 1.0638 - val_acc: 0.4384
Epoch 7/40
1100/1100 [==============================] - 9s 8ms/step - loss: 0.6246 - acc: 0.5900 - val_loss: 0.8829 - val_acc: 0.4529
Epoch 8/40
1100